In [1]:
import sys
sys.path.insert(0, '../scripts')
import compare_methods as cm
import numpy as np
from sage.rings.polynomial.multi_polynomial_ideal import MPolynomialIdeal
from sage.misc.sage_eval import sage_eval
from sage.rings.rational_field import QQ

In [125]:
result = cm.sample_format_inputs({'H11': 3, 'VOLFORMPARITY': -1})

In [126]:
k = result['DRESVERTS'].shape[0]
R = cm.build_ring(k)
P_monoms = cm.compute_monomials(R, result['NVERTS'], result['DRESVERTS'])
srideal = cm.compute_srideal(R, result['TRIANG'])
fixed_loci = cm.compute_fixed_loci(R, P_monoms, srideal, result['INVOL'])

In [127]:
X = np.array(R.gens())
X_sub = dict(zip(X, X[result['INVOL'].array_form]))
P_diff = [x - x.subs(X_sub) for x in P_monoms]
hysurf_diff_ideal = R.ideal(*P_diff).radical()
srideal_intersection = MPolynomialIdeal.intersection(*srideal).radical()
hysurf_diff_ideal = hysurf_diff_ideal.quotient(srideal_intersection)
fixed_loci1 = hysurf_diff_ideal.minimal_associated_primes()

In [128]:
result['OPLANES']

[{'OIDEAL': ['x2*x3-x4*x5'], 'ODIM': 7}]

In [136]:
a = MPolynomialIdeal.intersection(*fixed_loci).radical()

In [140]:
(a + R.ideal(*P_monoms)).radical()#.quotient(srideal_intersection)

Ideal (x6, x3*x4*x5, x2*x4*x5, x1*x3*x5, x1*x2*x5, x0*x3*x4, x0*x2*x4, x0*x1*x3, x0*x1*x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [146]:
fixed_loci

[Ideal (x5, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x1*x2 - x3*x4) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [147]:
fixed_loci1

[Ideal (x5, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x1*x2 - x3*x4) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [143]:
set(x.reduce(fixed_loci[0]) for x in P_monoms) - set([0])

{x6^2}

In [145]:
set(x.reduce(fixed_loci[2]) for x in P_monoms) - set([0])

IndexError: list index out of range

In [170]:
(fixed_loci[0] + R.ideal(*P_monoms) + srideal[0]).radical()#.quotient(srideal_intersection)

Ideal (x6, x5, x4, x1, x0) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field

In [162]:
srideal

[Ideal (x1, x4) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x2, x3) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field,
 Ideal (x0, x5, x6) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field]

In [175]:
P_monoms[4], P_monoms[4].reduce(hysurf_diff_ideal)

(x0^4*x2^4*x4^4, x0^4*x2^4*x4^4)

In [172]:
P_monoms[0]

x0^4*x3^4*x4^4

In [197]:
(fixed_loci[2] + R.ideal(*[x + x.subs(X_sub) for x in P_monoms])).radical()

IndexError: list index out of range

In [194]:
R.ideal(*[x + x.subs(X_sub) for x in P_monoms)).radical()

Ideal (x6, x3*x4*x5, x2*x4*x5, x1*x3*x5, x1*x2*x5, x0*x3*x4, x0*x2*x4, x0*x1*x3, x0*x1*x2) of Multivariate Polynomial Ring in x0, x1, x2, x3, x4, x5, x6 over Rational Field